Let's set up a study workflow using Jupyter Notebooks, LLMs, and langchain.

In [ ]:
# !pip install langchain
# !pip install langchain-openai
# !pip install pypdf
# !pip install chromadb
# !pip install langchainhub

In [1]:
import os
from langchain.document_loaders import PyPDFLoader
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

In [2]:
pdf_path = "./assets-resources/Augmenting-Human-Intellect_ A-Conceptual-Framework-1962-Doug Engelbart.pdf"

loader = PyPDFLoader(pdf_path) # LOAD
pdf_docs = loader.load_and_split() # SPLIT
embeddings = OpenAIEmbeddings() # EMBED
vectordb = Chroma.from_documents(pdf_docs, embedding=embeddings) # STORE
retriever = vectordb.as_retriever()
llm = ChatOpenAI(model="gpt-4-1106-preview")
pdf_qa = RetrievalQA.from_llm(llm=llm, retriever=retriever) # RETRIEVE
pdf_qa

RetrievalQA(combine_documents_chain=StuffDocumentsChain(llm_chain=LLMChain(prompt=ChatPromptTemplate(input_variables=['context', 'question'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template="Use the following pieces of context to answer the user's question. \nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\n----------------\n{context}")), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], template='{question}'))]), llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x1374efdc0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x167467e50>, model_name='gpt-4-1106-preview', openai_api_key=SecretStr('**********'), openai_api_base='https://api.openai.com/v1', openai_proxy='')), document_prompt=PromptTemplate(input_variables=['page_content'], template='Context:\n{page_content}'), document_variable_name='context'), re

In [3]:
query = "What is augmentation according to Douglas Engelbart?"
result = pdf_qa.invoke({"query": query, "chat_history": []})

In [4]:
print(result["result"])

According to Douglas Engelbart, augmentation refers to the ways in which human intellect can be improved and enhanced through the use of tools, techniques, and methodologies. Engelbart's vision of augmentation is closely linked to the development of computer-based systems that assist with symbol manipulation and decision-making processes, thereby extending the capabilities of the human mind.

In his seminal work "Augmenting Human Intellect: A Conceptual Framework," Engelbart explores the concept of augmentation in depth. He discusses how augmentation means can be redesigned to leverage the capabilities of computers, creating a symbiotic relationship between humans and machines that enhances cognitive functions such as learning, analysis, problem-solving, and decision-making.

Engelbart's approach to augmentation encompasses not just the physical tools or artifacts that people use, but also the language and methodology that shape how they think and work. His idea of augmentation is holi

In [5]:
def ask_pdf(pdf_qa,query):
    print("QUERY: ",query)
    result = pdf_qa.invoke({"query": query, "chat_history": []})
    answer = result["result"]
    print("ANSWER", answer)
    return answer


ask_pdf(pdf_qa,"What are the four ways through which human capabilities are augmented according to Douglas Engelbart?")

QUERY:  What are the four ways through which human capabilities are augmented according to Douglas Engelbart?
ANSWER Douglas Engelbart proposed a framework for augmenting human intellect which involves improving human capabilities through four main approaches:

1. **Artifacts**: Physical objects and technologies that serve as tools to extend human capabilities. These can range from simple tools like pencils and paper to complex computers and software that help us process and manipulate information more efficiently.

2. **Language**: The means by which humans communicate, including natural languages, mathematics, and various symbolic systems. Language allows for the expression of complex ideas and the ability to share knowledge and collaborate with others.

3. **Methodology**: The strategies, procedures, and techniques that people use to organize and approach problems. Methodologies guide how we use artifacts and language to effectively tackle tasks and make decisions.

4. **Training**:

"Douglas Engelbart proposed a framework for augmenting human intellect which involves improving human capabilities through four main approaches:\n\n1. **Artifacts**: Physical objects and technologies that serve as tools to extend human capabilities. These can range from simple tools like pencils and paper to complex computers and software that help us process and manipulate information more efficiently.\n\n2. **Language**: The means by which humans communicate, including natural languages, mathematics, and various symbolic systems. Language allows for the expression of complex ideas and the ability to share knowledge and collaborate with others.\n\n3. **Methodology**: The strategies, procedures, and techniques that people use to organize and approach problems. Methodologies guide how we use artifacts and language to effectively tackle tasks and make decisions.\n\n4. **Training**: The process of learning and acquiring skills in the use of artifacts, language, and methodologies. Training

In [6]:
quiz_questions = ask_pdf(pdf_qa, "Quiz me on the essay with 5 fundamental questions.")

quiz_questions

QUERY:  Quiz me on the essay with 5 fundamental questions.
ANSWER Certainly! Here are five fundamental questions based on the essay "Augmenting Human Intellect: A Conceptual Framework" by Douglas Engelbart:

1. **Conceptual Framework**: What is the main thesis or idea presented by Douglas Engelbart in the essay "Augmenting Human Intellect: A Conceptual Framework"?

2. **Symbol-Structuring Capabilities**: According to Engelbart, how does the capability to freely manipulate symbol structures on a computer enhance human intellect compared to traditional pencil-and-paper techniques?

3. **Statement Analysis**: How does Engelbart suggest handling complex statements for better understanding, and what are the benefits of such methods?

4. **Argument Structuring**: Describe the process Engelbart introduces for tracking the basis upon which a given statement in an argument is made. What are 'antecedent links' and how do they function in this process?

5. **Visualization and Comprehension**: Exp

'Certainly! Here are five fundamental questions based on the essay "Augmenting Human Intellect: A Conceptual Framework" by Douglas Engelbart:\n\n1. **Conceptual Framework**: What is the main thesis or idea presented by Douglas Engelbart in the essay "Augmenting Human Intellect: A Conceptual Framework"?\n\n2. **Symbol-Structuring Capabilities**: According to Engelbart, how does the capability to freely manipulate symbol structures on a computer enhance human intellect compared to traditional pencil-and-paper techniques?\n\n3. **Statement Analysis**: How does Engelbart suggest handling complex statements for better understanding, and what are the benefits of such methods?\n\n4. **Argument Structuring**: Describe the process Engelbart introduces for tracking the basis upon which a given statement in an argument is made. What are \'antecedent links\' and how do they function in this process?\n\n5. **Visualization and Comprehension**: Explain how Engelbart\'s proposed system allows for the 

In [7]:
llm = ChatOpenAI(model="gpt-4-1106-preview", temperature=0.0)

In [8]:
from langchain_core.prompts.chat import SystemMessagePromptTemplate, HumanMessagePromptTemplate

template = f"You take in text and spit out Python code doing what the user wants"
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_message_prompt = HumanMessagePromptTemplate.from_template("Return ONLY a PYTHON list containing the questions in this text: {questions}")

In [9]:
from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt,human_message_prompt])

In [10]:
quiz_chain = chat_prompt | llm

In [11]:
quiz_chain.invoke({"questions": quiz_questions})

AIMessage(content='```python\nquestions = [\n    "What is the main thesis or idea presented by Douglas Engelbart in the essay \'Augmenting Human Intellect: A Conceptual Framework\'?",\n    "According to Engelbart, how does the capability to freely manipulate symbol structures on a computer enhance human intellect compared to traditional pencil-and-paper techniques?",\n    "How does Engelbart suggest handling complex statements for better understanding, and what are the benefits of such methods?",\n    "Describe the process Engelbart introduces for tracking the basis upon which a given statement in an argument is made. What are \'antecedent links\' and how do they function in this process?",\n    "Explain how Engelbart\'s proposed system allows for the visualization of an argument\'s structure and how this aids in comprehending the relationships between different components of the argument."\n]\n```')

In [12]:
import re

def extract_python_code(markdown_string):
    pattern = r'```python\n(.*?)\n```'
    matches = re.findall(pattern, markdown_string, re.DOTALL)

    if matches:
        python_code = matches[0]
        return python_code
    else:
        return None

In [13]:
from langchain_core.runnables import RunnableLambda


quiz_chain = chat_prompt | llm | RunnableLambda(lambda x: x.content) | extract_python_code

Disclaimer: We haven't discussed runnable at length, but essentially they make up the core of the LCEL interface. 

`RunnableLambda` allows you to take in an output from part of the chain and pass it along after performing some transformation defined withint its lambda function.

In [14]:
questions_list = quiz_chain.invoke({"questions": quiz_questions})

In [15]:
questions_list

'questions = [\n    "What is the main thesis or idea presented by Douglas Engelbart in the essay \'Augmenting Human Intellect: A Conceptual Framework\'?",\n    "According to Engelbart, how does the capability to freely manipulate symbol structures on a computer enhance human intellect compared to traditional pencil-and-paper techniques?",\n    "How does Engelbart suggest handling complex statements for better understanding, and what are the benefits of such methods?",\n    "Describe the process Engelbart introduces for tracking the basis upon which a given statement in an argument is made. What are \'antecedent links\' and how do they function in this process?",\n    "Explain how Engelbart\'s proposed system allows for the visualization of an argument\'s structure and how this aids in comprehending the relationships between different components of the argument."\n]'

In [16]:
exec(questions_list)

In [17]:
questions

["What is the main thesis or idea presented by Douglas Engelbart in the essay 'Augmenting Human Intellect: A Conceptual Framework'?",
 'According to Engelbart, how does the capability to freely manipulate symbol structures on a computer enhance human intellect compared to traditional pencil-and-paper techniques?',
 'How does Engelbart suggest handling complex statements for better understanding, and what are the benefits of such methods?',
 "Describe the process Engelbart introduces for tracking the basis upon which a given statement in an argument is made. What are 'antecedent links' and how do they function in this process?",
 "Explain how Engelbart's proposed system allows for the visualization of an argument's structure and how this aids in comprehending the relationships between different components of the argument."]

In [18]:
# the questions variable was created within the string inside the `questions_list` variable.
answers = []
for q in questions:
    answers.append(ask_pdf(pdf_qa,q))

QUERY:  What is the main thesis or idea presented by Douglas Engelbart in the essay 'Augmenting Human Intellect: A Conceptual Framework'?
ANSWER The main thesis presented by Douglas Engelbart in the essay "Augmenting Human Intellect: A Conceptual Framework" is that the intellectual effectiveness of a human being can be significantly enhanced through the systematic redesign of the means by which individuals approach and solve problems. Engelbart proposes that by developing and utilizing tools and techniques that augment human intellect, we can improve our ability to think, learn, and innovate. This includes leveraging disciplines such as engineering, mathematics, and various sciences to create systems that enhance our cognitive processes. An important aspect of this thesis is the integration of computers and digital displays into the workflow to assist humans in real-time, enabling a symbiotic relationship between human thought processes and machine capabilities. The goal is to test thi

In [91]:
evaluations = []

for q,a in zip(questions, answers):
    # Check for results
    evaluations.append(ask_pdf(pdf_qa,f"Is this: {a} the correct answer to this question: {q} according to the paper? Return ONLY '''YES''' or '''NO'''. Output:"))

evaluations

QUERY:  Is this: The primary goal that Doug Engelbart envisions achieving through augmenting human intellect is to develop ways to increase the capability of a human to approach a complex problem situation, to gain comprehension to suit his particular needs, and to derive solutions to problems. Engelbart's focus was on enhancing human ability to solve problems, make decisions, and understand complex issues more effectively through the use of advanced tools, methodologies, and processes. This augmentation is not just about automating tasks but rather about empowering humans to do what was previously impossible or extremely difficult, thereby expanding both the intellectual and creative capacity of individuals and groups. the correct answer to this question: What is the primary goal that Doug Engelbart envisions achieving through augmenting human intellect? according to the paper? Return ONLY '''YES''' or '''NO'''. Output:
ANSWER YES
QUERY:  Is this: Engelbart suggests that computers can

['YES', 'YES', 'YES', 'YES', 'YES']

In [93]:
scores = []

yes_count = evaluations.count('YES')
scores.append(yes_count)
print(scores)

[5]
